In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.feature_selection import RFE

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
#Mount Google Drive
drive.mount('/content/gdrive')

#Set the data path for dataset and model location (ex: model_loc = "/content/gdrive/My Drive/Dataset/")
dataset_dir = "/content/gdrive/My Drive/Bio Modelling/data/"
model_loc = "/content/gdrive/My Drive/Bio Modelling/data/"

print(os.listdir(dataset_dir))

Mounted at /content/gdrive
['BRCA_miRNA.csv', 'BRCA_label.csv', 'BRCA_DNA_Methylation.csv', 'BRCA_mRNA.csv', 'BRCA_Copy Nunber Variation.csv']


In [ ]:
dna = pd.read_csv(dataset_dir+'BRCA_DNA_Methylation.csv',index_col=0)
dna

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,-1.211396,0.974531,0.633283,0.354642,-1.939963,1.328608,0.946010,0.930281,-0.519615,0.554409,...,-0.760042,1.159665,0.333349,0.775567,-0.778884,0.517100,0.177384,-0.380611,0.811548,-0.819564
A1CF,-1.801239,-1.091976,-0.258825,0.344929,-0.018208,-1.007907,1.246215,0.360814,-0.426866,-1.357345,...,-0.582298,-2.142905,-0.306365,0.816486,-0.499731,0.824666,0.807188,-0.027147,0.644539,-2.004627
A2M,-0.696763,-1.597015,-0.873714,-1.080573,-0.788607,-0.583001,-0.654375,0.302439,-1.561816,-0.836623,...,-1.001059,-2.915892,0.635853,0.027964,-1.145160,-0.326970,0.569131,-2.613169,-0.507579,-0.339844
A2ML1,0.832338,0.758754,0.753608,0.548382,0.589277,0.870132,-1.878898,0.160075,0.066117,-0.338768,...,0.892903,-2.480427,0.319169,-0.098838,0.830086,0.617726,0.551416,0.101744,0.269641,-0.094397
A3GALT2,0.857098,0.450506,0.821330,0.497375,0.400632,1.183863,-3.316770,0.227579,0.051009,0.300039,...,1.161783,-1.863239,0.350236,-0.879849,-0.098983,0.524779,0.041604,0.537643,-0.322718,0.007108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,1.079773,1.107008,1.143368,1.327138,0.825476,1.014806,-0.726589,0.116715,-0.967701,-0.634965,...,0.501544,1.248136,-0.609775,0.259632,1.086309,1.867683,0.048509,-0.122038,0.896755,-0.756927
ZYX,0.780058,0.404568,-0.583163,0.047150,0.272223,1.304881,-0.084193,-0.078364,-0.585443,-0.713563,...,0.578764,1.092554,0.706154,-0.837710,1.021682,1.258202,-0.639635,-0.988974,1.156913,-1.043962
ZZEF1,0.747015,0.107306,0.034379,0.014622,0.298979,1.151613,-0.621557,-0.887355,-0.625655,0.094450,...,0.277770,0.629951,0.062128,-0.269128,0.820279,0.573421,-0.542603,-0.854843,0.898579,-0.878013
ZZZ3,0.378339,0.271888,-0.170242,0.484366,0.029261,0.970602,-0.468274,-0.221699,-0.738370,-0.381204,...,0.781932,0.620894,-0.375671,-0.597399,0.227439,1.354182,-0.548846,-0.365477,-0.098397,-0.763912


In [ ]:
cnv = pd.read_csv(dataset_dir+'BRCA_Copy Nunber Variation.csv',index_col=0)
cnv

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,0.495252,1.028904,0.003059,-0.258705,-0.083232,-0.486242,-0.111192,-0.102997,0.687116,-0.213873,...,-0.493956,2.097171,-0.594226,0.478862,-1.755051,-0.835261,-0.148311,0.557439,-0.234602,-0.838154
A1CF,-0.938762,-1.575977,0.405308,0.096910,0.060782,2.050968,0.154506,0.018894,1.131535,0.145081,...,-0.299975,6.501006,0.255559,-0.184784,0.371274,0.483323,0.061306,0.627836,0.070730,-0.463861
A2M,0.750838,0.010864,0.103201,-0.169130,-2.090765,-0.216362,-0.062751,-0.056368,0.857217,-0.016369,...,1.139760,-1.938856,0.036395,-0.308274,0.118520,0.266599,-0.089133,0.344469,0.226601,-0.590817
A2ML1,0.748593,0.005894,0.098571,-0.174763,-2.103474,-0.222169,-0.067992,-0.061585,0.855364,-0.021439,...,1.138948,-1.951005,0.031519,-0.314419,0.113946,0.262571,-0.094471,0.340728,0.222425,-0.598003
A3GALT2,1.093743,-1.575680,-0.706498,0.199712,0.376570,-2.334753,0.400443,0.225047,1.226264,0.390699,...,-2.541330,-1.587373,0.371698,0.096424,-1.494803,-2.569588,0.046241,-1.244865,-0.252418,-0.264599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,1.267331,-1.793233,-0.872324,0.058733,0.242915,-2.475162,0.263210,0.085117,1.176508,0.207398,...,-0.169592,0.192176,0.237841,-0.048833,-1.672982,-2.900353,-0.073695,-0.277665,-0.435463,-0.424808
ZYX,0.963246,1.446230,0.039191,-0.257130,-0.185487,-1.970238,-0.112381,-0.106533,0.704939,-0.262491,...,-0.542729,-0.489606,-0.134313,0.310169,-0.002235,1.155757,-0.286372,0.253634,-0.327799,-0.589030
ZZEF1,-0.580418,-0.790000,-1.421040,-1.377441,-0.999581,1.646581,0.848410,-0.727278,-0.501251,0.866003,...,-1.375911,-0.695152,0.920311,-0.919650,-0.690180,0.438043,0.811313,-0.470272,0.723732,-2.171404
ZZZ3,1.715811,-1.268621,-0.811368,0.098802,0.204322,-1.930651,0.307432,0.230822,-1.278740,0.290087,...,-2.491497,-0.100674,0.289605,-0.009609,-1.555790,-2.902013,-0.051046,-0.247150,-0.350742,-0.339178


In [ ]:
mirna = pd.read_csv(dataset_dir+'BRCA_miRNA.csv',index_col=0)
mirna

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
hsa.let.7a.1,0.068317,-0.301684,-0.150810,0.107831,0.395211,1.135212,1.813887,0.473642,-0.470612,0.791968,...,-3.038971,-0.474111,0.165775,0.706495,-0.086149,0.511958,1.225298,0.667662,-0.211878,0.474240
hsa.let.7a.2,0.068932,-0.318009,-0.122747,0.097594,0.412879,1.140183,1.790792,0.442224,-0.461217,0.841821,...,-3.037340,-0.477443,0.172506,0.706552,-0.074380,0.504240,1.219548,0.666012,-0.210525,0.488117
hsa.let.7a.3,0.073899,-0.301310,-0.126333,0.095545,0.418441,1.143605,1.822272,0.431312,-0.470734,0.805374,...,-3.017967,-0.474484,0.164726,0.727433,-0.102883,0.496009,1.210610,0.675998,-0.220135,0.481577
hsa.let.7b,0.524562,0.419859,-0.958939,0.615389,0.500594,1.159087,2.113821,0.812876,-0.129307,0.867411,...,-3.509653,-0.855498,-0.359797,0.997679,-0.342075,0.422382,0.050804,-0.102655,0.177655,-0.078426
hsa.let.7c,-1.656853,-0.715963,-0.971038,0.711952,0.426323,-1.058870,2.135373,1.031809,0.864211,0.071249,...,-2.454470,0.188937,-0.001077,1.302563,0.506728,1.028977,1.186182,1.534247,1.070328,-0.155655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hsa.mir.95,0.943346,-0.828843,-0.487207,-0.539524,-0.490655,1.716711,-0.311971,-0.207033,-0.411471,1.331634,...,-1.290823,-1.028967,1.079692,-0.906212,0.475039,-0.867968,-0.019299,-1.125255,-0.477013,0.266220
hsa.mir.96,1.552613,0.599794,1.261482,0.856983,1.188748,0.375870,0.297224,0.066368,0.268172,0.155349,...,1.077028,-0.538255,0.875546,0.880267,0.479055,0.988195,-1.079726,0.219581,0.480704,0.198107
hsa.mir.98,-0.143572,1.103662,1.940594,-0.831098,-0.885905,-0.922881,0.390134,-0.330246,-0.481707,0.490018,...,1.152087,-0.648284,0.562151,-0.826311,0.280432,0.034551,0.419871,0.277618,-0.187117,0.914841
hsa.mir.99a,-1.251868,-0.627587,-0.848552,0.936474,0.449284,-1.316179,2.106744,1.078635,1.255710,-0.062945,...,-2.406554,0.465270,0.281323,1.373466,0.740620,0.989033,1.595339,1.039647,1.008012,0.094512


In [ ]:
data=pd.concat([dna,mirna,cnv])
data

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,-1.211396,0.974531,0.633283,0.354642,-1.939963,1.328608,0.946010,0.930281,-0.519615,0.554409,...,-0.760042,1.159665,0.333349,0.775567,-0.778884,0.517100,0.177384,-0.380611,0.811548,-0.819564
A1CF,-1.801239,-1.091976,-0.258825,0.344929,-0.018208,-1.007907,1.246215,0.360814,-0.426866,-1.357345,...,-0.582298,-2.142905,-0.306365,0.816486,-0.499731,0.824666,0.807188,-0.027147,0.644539,-2.004627
A2M,-0.696763,-1.597015,-0.873714,-1.080573,-0.788607,-0.583001,-0.654375,0.302439,-1.561816,-0.836623,...,-1.001059,-2.915892,0.635853,0.027964,-1.145160,-0.326970,0.569131,-2.613169,-0.507579,-0.339844
A2ML1,0.832338,0.758754,0.753608,0.548382,0.589277,0.870132,-1.878898,0.160075,0.066117,-0.338768,...,0.892903,-2.480427,0.319169,-0.098838,0.830086,0.617726,0.551416,0.101744,0.269641,-0.094397
A3GALT2,0.857098,0.450506,0.821330,0.497375,0.400632,1.183863,-3.316770,0.227579,0.051009,0.300039,...,1.161783,-1.863239,0.350236,-0.879849,-0.098983,0.524779,0.041604,0.537643,-0.322718,0.007108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,1.267331,-1.793233,-0.872324,0.058733,0.242915,-2.475162,0.263210,0.085117,1.176508,0.207398,...,-0.169592,0.192176,0.237841,-0.048833,-1.672982,-2.900353,-0.073695,-0.277665,-0.435463,-0.424808
ZYX,0.963246,1.446230,0.039191,-0.257130,-0.185487,-1.970238,-0.112381,-0.106533,0.704939,-0.262491,...,-0.542729,-0.489606,-0.134313,0.310169,-0.002235,1.155757,-0.286372,0.253634,-0.327799,-0.589030
ZZEF1,-0.580418,-0.790000,-1.421040,-1.377441,-0.999581,1.646581,0.848410,-0.727278,-0.501251,0.866003,...,-1.375911,-0.695152,0.920311,-0.919650,-0.690180,0.438043,0.811313,-0.470272,0.723732,-2.171404
ZZZ3,1.715811,-1.268621,-0.811368,0.098802,0.204322,-1.930651,0.307432,0.230822,-1.278740,0.290087,...,-2.491497,-0.100674,0.289605,-0.009609,-1.555790,-2.902013,-0.051046,-0.247150,-0.350742,-0.339178


In [ ]:
label = pd.read_csv(dataset_dir+'BRCA_label.csv')
label

,Label
0,LumA
1,Her2
2,LumB
3,LumA
4,LumA
...,...
666,LumA
667,LumA
668,LumA
669,LumA


In [ ]:
#replace label with numerical data
label['Label'].replace('Normal', 0 , inplace=True)
label['Label'].replace('LumA', 1 , inplace=True)
label['Label'].replace('LumB', 2 , inplace=True)
label['Label'].replace('Basal', 3 , inplace=True)
label['Label'].replace('Her2', 4 , inplace=True)

In [ ]:
# Transpose data to match with label
data = data. T
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,-0.837399,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,-1.537275,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,4.165544,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,0.080744,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,0.045801,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.517100,0.824666,-0.326970,0.617726,0.524779,0.887775,0.553903,0.059472,0.150010,-0.999820,...,0.454483,2.317018,2.260655,-0.081225,-2.879783,-2.900353,1.155757,0.438043,-2.902013,0.247469
TCGA.XX.A899.01,0.177384,0.807188,0.569131,0.551416,0.041604,0.890293,0.012863,-1.045639,0.102895,0.209390,...,0.046307,-0.281173,-0.263396,-0.182627,-0.075447,-0.073695,-0.286372,0.811313,-0.051046,-0.178246
TCGA.XX.A89A.01,-0.380611,-0.027147,-2.613169,0.101744,0.537643,-0.910491,-1.371328,-0.607679,0.085117,-1.533629,...,0.594256,-0.279522,-0.261792,-0.686897,-0.277806,-0.277665,0.253634,-0.470272,-0.247150,-0.427626
TCGA.Z7.A8R5.01,0.811548,0.644539,-0.507579,0.269641,-0.322718,0.886293,0.555758,-0.144756,-0.187391,-0.931151,...,0.012883,-0.345571,-0.325956,-0.844207,-0.434358,-0.435463,-0.327799,0.723732,-0.350742,0.076841


In [ ]:
# Ensure the number of rows in data match with label
if data.shape[0] == label.shape[0]:
    # Add the new column to the transposed DataFrame
    data['Label'] = label['Label'].values
else:
    raise ValueError("The number of rows in df2 must match the number of columns in the transposed df1.")


In [ ]:
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183,1
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052,4
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237,2
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601,1
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.517100,0.824666,-0.326970,0.617726,0.524779,0.887775,0.553903,0.059472,0.150010,-0.999820,...,2.317018,2.260655,-0.081225,-2.879783,-2.900353,1.155757,0.438043,-2.902013,0.247469,1
TCGA.XX.A899.01,0.177384,0.807188,0.569131,0.551416,0.041604,0.890293,0.012863,-1.045639,0.102895,0.209390,...,-0.281173,-0.263396,-0.182627,-0.075447,-0.073695,-0.286372,0.811313,-0.051046,-0.178246,1
TCGA.XX.A89A.01,-0.380611,-0.027147,-2.613169,0.101744,0.537643,-0.910491,-1.371328,-0.607679,0.085117,-1.533629,...,-0.279522,-0.261792,-0.686897,-0.277806,-0.277665,0.253634,-0.470272,-0.247150,-0.427626,1
TCGA.Z7.A8R5.01,0.811548,0.644539,-0.507579,0.269641,-0.322718,0.886293,0.555758,-0.144756,-0.187391,-0.931151,...,-0.345571,-0.325956,-0.844207,-0.434358,-0.435463,-0.327799,0.723732,-0.350742,0.076841,1


In [ ]:
# print the total number of data with missing values across all columns
print(data.isna().sum().sum())

0


In [ ]:
# print the total number of data with duplicated values across all columns
print(data.duplicated().sum())

0


In [ ]:
print(data["Label"].value_counts()[0])
print(data["Label"].value_counts()[1])
print(data["Label"].value_counts()[2])
print(data["Label"].value_counts()[3])
print(data["Label"].value_counts()[4])

31
353
132
113
42


In [ ]:
# Separate features and target
y = data['Label'].values
x = data.drop(['Label'],axis=1)

In [ ]:
# Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler.fit_transform(x)

# Apply the scaler to the DataFrame
x = pd.DataFrame(scaler.fit_transform(X_normalized), index = x.index, columns = x.columns)
x

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,0.040905,-0.420021,0.335390,0.932236,0.589542,-0.694817,0.352448,-0.529989,0.776087,-0.701358,...,-0.411802,0.206165,-0.012023,0.354060,0.157600,0.236162,-0.009469,-0.397939,0.321668,-0.302060
TCGA.3C.AALI.01,0.764607,-0.116690,0.027662,0.910876,0.439674,0.553878,0.542636,-0.903589,0.678537,0.318271,...,-0.521860,0.526421,0.250301,0.461315,-0.545799,-0.514974,0.082529,-0.438278,-0.377547,-0.164382
TCGA.3C.AALJ.01,0.651629,0.239624,0.274904,0.909382,0.576358,-0.515340,0.334061,-0.654200,-0.090193,0.010108,...,0.374930,-0.172138,-0.321893,0.047760,-0.334150,-0.288961,-0.185481,-0.559735,-0.270418,-0.358196
TCGA.3C.AALK.01,0.559379,0.497831,0.204195,0.849807,0.456950,0.428291,0.268593,-0.746226,0.761610,0.573245,...,-0.267421,-0.171604,-0.321456,0.089763,-0.120168,-0.060457,-0.241924,-0.551343,-0.057177,-0.198817
TCGA.5L.AAT0.01,-0.200304,0.342528,0.303996,0.861678,0.421291,0.544444,0.339458,-0.581956,0.724191,0.205122,...,-0.272916,-0.170136,-0.320254,0.103252,-0.077838,-0.015254,-0.228277,-0.478616,-0.032455,-0.194724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.613164,0.703000,0.461795,0.869937,0.467051,0.177660,0.614690,-0.663773,0.818125,0.377529,...,-0.208649,0.254070,0.027216,0.127049,-0.800245,-0.786688,0.027200,-0.201914,-0.760230,-0.175492
TCGA.XX.A899.01,0.500693,0.695526,0.768104,0.850687,0.288956,0.179048,0.440127,-0.936369,0.810228,0.685782,...,-0.272836,-0.165866,-0.316756,0.106761,-0.150603,-0.092958,-0.247494,-0.130070,-0.092284,-0.223962
TCGA.XX.A89A.01,0.315956,0.338705,-0.319684,0.720152,0.471793,-0.813569,-0.006471,-0.828338,0.807248,0.241451,...,-0.186669,-0.165599,-0.316537,0.005867,-0.197481,-0.143017,-0.144634,-0.376739,-0.138229,-0.252355
TCGA.Z7.A8R5.01,0.710648,0.625966,0.400058,0.768891,0.154669,0.176843,0.615288,-0.714150,0.761575,0.395035,...,-0.278092,-0.176274,-0.325281,-0.025607,-0.233747,-0.181745,-0.255384,-0.146927,-0.162499,-0.194919


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))
print(y_train.tolist().count(2))
print(y_train.tolist().count(3))
print(y_train.tolist().count(4))

In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=38000, step=10)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Transform the training and test sets to include only the selected features
x_train_rfe = rfe.transform(X_train)
x_test_rfe = rfe.transform(X_test)

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features = X_train.columns[rfe.support_]

X_train_rfe = pd.DataFrame(x_train_rfe, columns=selected_features, index=X_train.index)
X_test_rfe = pd.DataFrame(x_test_rfe, columns=selected_features, index=X_test.index)

In [ ]:
oversample = SMOTE()
X_resampled, y_resampled = oversample.fit_resample(X_train_rfe, y_train)
y_resampled = pd.DataFrame(y_resampled, columns=['Label'])

In [ ]:
print(y_resampled["Label"].value_counts()[0])
print(y_resampled["Label"].value_counts()[1])
print(y_resampled["Label"].value_counts()[2])
print(y_resampled["Label"].value_counts()[3])
print(y_resampled["Label"].value_counts()[4])

247
247
247
247
247


In [ ]:
# Define the model
model = Sequential()

# Add convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(38000, 1)))
model.add(MaxPooling1D(pool_size=2))

# Add more convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Add another convolutional layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(5, activation='softmax'))  # 5 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 36998, 32)         128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 18499, 32)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 18497, 64)         6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 9248, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 9246, 128)         24704     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 4623, 128)         0

In [ ]:
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_resampled, y_resampled, epochs=250, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
31/31 [==============================] - 13s 197ms/step - loss: 4.0314 - accuracy: 0.4312 - val_loss: 2.5599 - val_accuracy: 0.0972
Epoch 2/100
31/31 [==============================] - 4s 129ms/step - loss: 0.7476 - accuracy: 0.7328 - val_loss: 3.2366 - val_accuracy: 0.1012
Epoch 3/100
31/31 [==============================] - 4s 128ms/step - loss: 0.5269 - accuracy: 0.8168 - val_loss: 2.0739 - val_accuracy: 0.1053
Epoch 4/100
31/31 [==============================] - 4s 127ms/step - loss: 0.4097 - accuracy: 0.8532 - val_loss: 1.8578 - val_accuracy: 0.1134
Epoch 5/100
31/31 [==============================] - 4s 130ms/step - loss: 0.3071 - accuracy: 0.8846 - val_loss: 1.5373 - val_accuracy: 0.2470
Epoch 6/100
31/31 [==============================] - 4s 129ms/step - loss: 0.3077 - accuracy: 0.8897 - val_loss: 1.0246 - val_accuracy: 0.5344
Epoch 7/100
31/31 [==============================] - 4s 127ms/step - loss: 0.2635 - accuracy: 0.8988 - val_loss: 1.3811 - val_accuracy: 0.336

In [ ]:
# Predict probabilities
y_pred_prob = model.predict(X_test_rfe)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 1s 72ms/step
Accuracy: 0.7920792079207921
